# Geocoding with Python and MapBox API


**Date**: 14.02.2024

**Course**: Methods of Spatial Analysis. Advanced Level. // HSE, Moscow, spring 2024


### Geocoding

0.  Get API for geocoding [here](https://docs.mapbox.com/api/search/geocoding/)
1.  Install all the needed libraries (requied only for the first time)
2.  Import libraries
3.  Geocoding basics (set parameters, get request)
4.  Geocoding addresses from a Data Table

### Fun

5. Create a spatial point layer form coordinates in a .csv file
6. View the result on a web map
7. Export a GeoDataFrame to any favorite GIS format
8. Create HTML page or PDF and share it with your friends who need to geocode addresses :)


# Geocoding


## 1. Install libraries


#### Installing libraries IN !!!TERMINAL!!!

```
pip install pandas
pip install geopandas
pip install requests

```


## 2. Import libraries


In [2]:
## For working with dataframes

import pandas as pd
import geopandas as gpd

# For working with requests
import requests
import json

/var/folders/ry/9bb7wrz54vq_kn2ytlj6ynzm0000gn/T/ipykernel_68703/1528870275.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 3. Geocoding basics


### Defining parameters for geocoding


In [55]:
API_KEY = 'pk.eyJ1IjoiYmVsa2FtaXIiLCJhIjoiY2t1d2h0cmM2MG8wYTJ3cWY3Y21zZmQ3ZCJ9.4r8LDCKXDdlEa0Rr8FnLUQ' 

params = {
    'access_token': API_KEY,
    'autocomplete': 'false', # Disable address autocompletion
    'limit': 1, # Limit the number of results to 1
    'types': 'address', # Specify the address type
    'country': 'RU', # Limit the search to Russia only
    'language': 'ru', # Response language
    }

sample_address = "Москва, Волхонка, д.12"

url = 'https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json'.format(sample_address)


#### Creating a get request


In [56]:
response = requests.get(url, params=params)

#### Exploring response


In [57]:
response
print(response.status_code)
print(response.json())

200
{'type': 'FeatureCollection', 'query': ['обл', 'иркутская', 'г', 'братск', 'ул', 'студенческая', 'д', '18'], 'features': [{'id': 'address.2535955410048640', 'type': 'Feature', 'place_type': ['address'], 'relevance': 0.42563, 'properties': {'accuracy': 'street', 'mapbox_id': 'dXJuOm1ieGFkcjplNjI5ODg5OC0wN2U4LTRiYTctOWY1YS1mZDIwYTI1MWVmZjA'}, 'text_ru': 'Иркутская Улица', 'place_name_ru': 'Россия, Иркутская область, Братск, 665776, Иркутская Улица', 'text': 'Иркутская Улица', 'place_name': 'Россия, Иркутская область, Братск, 665776, Иркутская Улица', 'center': [101.908205271792, 56.2792203], 'geometry': {'type': 'Point', 'coordinates': [101.908205271792, 56.2792203]}, 'context': [{'id': 'postcode.303509186', 'mapbox_id': 'dXJuOm1ieHBsYzpFaGN1d2c', 'text_ru': '665776', 'text': '665776'}, {'id': 'place.18204866', 'mapbox_id': 'dXJuOm1ieHBsYzpBUlhJd2c', 'wikidata': 'Q132732', 'text_ru': 'Братск', 'language_ru': 'ru', 'text': 'Братск', 'language': 'ru'}, {'id': 'region.591042', 'mapbox_i

#### Saving response data to a variable


In [58]:
data = response.json()

#### Extracting coordinates from data


In [60]:
coordinates = data['features'][0]['center']
longitude, latitude = coordinates
print(longitude, latitude)

101.908205271792 56.2792203


## 4.Geocoding addresses from a Data Table


#### Reading data


In [61]:
df = pd.read_csv('./df_sample.csv', sep=';', on_bad_lines='skip')
address_column = 'address' 

In [62]:
df.head()

,id,address
0,6482827,"обл. Иркутская, г. Братск, ул. 25-летия Братск..."
1,6482828,"обл. Иркутская, г. Братск, ул. 25-летия Братск..."
2,6482829,"обл. Иркутская, г. Братск, ул. 25-летия Братск..."
3,6690904,"обл. Иркутская, г. Братск, ул. 25-летия Братск..."
4,9155620,"обл. Иркутская, г. Братск, ул. 25-летия Братск..."


#### Creating columns for coordinates


In [63]:
df['longitude'] = ''
df['latitude'] = ''

#### Geocoding addresses from the table


In [64]:
for index, row in df.iterrows():
    address = row['address']
    url = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json'
    try:
        response = requests.get(url, params=params)
        data = response.json()
        coordinates = data['features'][0]['center']
        longitude, latitude = coordinates
        df.at[index, 'longitude'] = longitude
        df.at[index, 'latitude'] = latitude
    except Exception as e:
            print(f"Error with {address}: {e}")

#### Saving result to a csv file


In [65]:
df.to_csv('./df_sample_geocoded.csv', index=False, index_label=False)

#### Function to geocode adresses from a csv file


In [ ]:
def geocode_addresses(csv_file, api_key):
    # Loading CSV file
    df = pd.read_csv(csv_file, delimiter=';', encoding='utf-8')
    
    ## Creating empty columns for coordinates
    df['longitude'] = ''
    df['latitude'] = ''

    # Defining parameters
    params = {
        'access_token': api_key,
        'autocomplete': 'false',
        'limit': 1
        }
    
    # Geocoding addresses
    for index, row in df.iterrows():
        address = row['address']
        url = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json'
 
        try:
            response = requests.get(url, params=params)
            data = response.json()
            coordinates = data['features'][0]['center']
            longitude, latitude = coordinates
            df.at[index, 'longitude'] = longitude
            df.at[index, 'latitude'] = latitude
        except Exception as e:
            print(f"Error with {address}: {e}")

    # Saving the results back to the CSV file
    df.to_csv(f'{csv_file}_geocoded', sep=';', index=False, encoding='utf-8')
    print("Geocoding completed. Results saved to file")


#### Geocoding using functions 'geocode_addresses'


In [ ]:
geocode_addresses('df_sample.csv', API_KEY)

# FUN


## 5.Creating a spatial point layer form a data frame with coordinates


#### Creating GeoDataFrame


In [70]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

In [71]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

## 6. View the result on a web map


Import library for a web-map


In [72]:
import folium

Create web-map box


In [74]:
gdf.explore()

## 11. Export a GeoDataFrame to any favorite GIS format


In [75]:
gdf.to_file('gdf_sample.gpkg')